In [3]:
import cv2
import imutils
import numpy as np
from imutils import contours
from matplotlib import pyplot as plt
import pytesseract

def compare_white_black(img):
    white_count = cv2.countNonZero(img)
    black_count = img.size - white_count  # Count of black pixels (0)
    print(black_count)
    print(white_count)
    return white_count > black_count

def showImage(name, image, cmap=None):
    plt.imshow(image, cmap=cmap)
    plt.axis('off')  # Hide axis
    plt.title(name)
    plt.show()

FIRST_NUMBER = {
    "3": "American Express",
    "4": "Visa",
    "5": "MasterCard",
    "6": "Discover Card"
}

# Load the reference image of the digits
ref = cv2.imread('Credit-Card0.png')
ref = cv2.cvtColor(ref, cv2.COLOR_BGR2GRAY)
ref = cv2.threshold(ref, 10, 255, cv2.THRESH_BINARY_INV)[1]

# Find contours in the reference image
refCnts = cv2.findContours(ref.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
refCnts = imutils.grab_contours(refCnts)
refCnts = contours.sort_contours(refCnts, method="left-to-right")[0]
digits = {}

# Store each digit's region of interest (ROI)
for (i, c) in enumerate(refCnts):
    (x, y, w, h) = cv2.boundingRect(c)
    roi = ref[y:y + h, x:x + w]
    roi = cv2.resize(roi, (57, 88))
    digits[i] = roi

# Load the input image containing only the PAN of the credit card
image = resizedpan.copy()  # Ensure `pan` is defined with the correct image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Threshold the gray image to get a binary image
group = cv2.threshold(gray, 55, 255, cv2.THRESH_BINARY_INV)[1]
showImage("Thresholded Image", group, cmap="gray")
# group = cv2.medianBlur(group, ksize=3)

# Invert the image if white pixels are more than black pixels
if compare_white_black(group):
    group = cv2.bitwise_not(group)
    showImage("Inverted Image", group, cmap="gray")


# Erode to remove thin parts
# Define the size of the kernel
kernel_size = 5  # Adjust this based on your desired thickness
kernelx = np.ones((kernel_size, kernel_size), np.uint8)
eroded = cv2.erode(group, kernelx, iterations=1)
dilated = cv2.dilate(eroded, kernel, iterations=1)

showImage("eroded",eroded)
showImage("dilated",dilated)
kerneleroded = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))  # 5x5 rectangular kernel

# Find contours in the binary image
digitCnts = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
digitCnts = imutils.grab_contours(digitCnts)

# Filter out very small contours
min_contour_area = 250  # Define the minimum contour area threshold
filteredCnts = [c for c in digitCnts if cv2.contourArea(c) > min_contour_area]
if filteredCnts:
# Sort the filtered contours from left to right
    filteredCnts = contours.sort_contours(filteredCnts, method="left-to-right")[0]
else:
    print("no contours found")
output = []

# Loop over each detected contour to identify digits
for c in filteredCnts:
    (x, y, w, h) = cv2.boundingRect(c)
    roi = group[y:y + h, x:x + w]
    roi = cv2.resize(roi, (57, 88))
    scores = []

    # Compare each ROI with reference digits to identify the digit
    for (digit, digitROI) in digits.items():
        result = cv2.matchTemplate(roi, digitROI, cv2.TM_CCOEFF)
        (_, score, _, _) = cv2.minMaxLoc(result)
        scores.append(score)

    # Append the recognized digit to the output list
    output.append(str(np.argmax(scores)))

    # Draw the contour on the original image
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw a rectangle around the detected digit
print("Credit Card #: {}".format("".join(output)))
    # Show the original image with contours drawn
showImage("Detected Contours", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
if len(output)==16:
# Print the recognized PAN
    print("Credit Card #: {}".format("".join(output)))
    # Show the original image with contours drawn
    showImage("Detected Contours", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

else:
    kernel = np.ones((3, 3), np.uint8)  # You can adjust the kernel size
    gradient = cv2.morphologyEx(sharpened, cv2.MORPH_GRADIENT, kernel)

    showImage("gradient pan",gradient)
  
    # Load the input image containing only the PAN of the credit card
    image = gradient  # Ensure `pan` is defined with the correct image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Threshold the gray image to get a binary image
    group = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)[1]
    showImage("Thresholded Image", group, cmap="gray")
    # group = cv2.medianBlur(group, ksize=3)

    # Invert the image if white pixels are more than black pixels
    if compare_white_black(group):
        group = cv2.bitwise_not(group)
        showImage("Inverted Image", group, cmap="gray")

    # Find contours in the binary image
    digitCnts = cv2.findContours(group.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    digitCnts = imutils.grab_contours(digitCnts)

    # Filter out very small contours
    min_contour_area = 16  # Define the minimum contour area threshold
    filteredCnts = [c for c in digitCnts if cv2.contourArea(c) > min_contour_area]

    # Sort the filtered contours from left to right
    filteredCnts = contours.sort_contours(filteredCnts, method="left-to-right")[0]

    output = []

    # Loop over each detected contour to identify digits
    for c in filteredCnts:
        (x, y, w, h) = cv2.boundingRect(c)
        roi = group[y:y + h, x:x + w]
        roi = cv2.resize(roi, (57, 88))
        scores = []

        # Compare each ROI with reference digits to identify the digit
        for (digit, digitROI) in digits.items():
            result = cv2.matchTemplate(roi, digitROI, cv2.TM_CCOEFF)
            (_, score, _, _) = cv2.minMaxLoc(result)
            scores.append(score)

        # Append the recognized digit to the output list
        output.append(str(np.argmax(scores)))

        # Draw the contour on the original image
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw a rectangle around the detected digit

    print("Credit Card #: {}".format("".join(output)))
    # Show the original image with contours drawn
    showImage("Detected Contours", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

